In [ ]:
# TODO - Define ResNet18 model.
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Layer, BatchNormalization, ReLU, Add, GlobalAvgPool2D, Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from enum import Enum


cifar100 = tf.keras.datasets.cifar100 
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

class BasicBlock_withResidule(Layer):
    def __init__(self, filters: int, strides: int, downsample=False):
        super().__init__()
        self.conv0 = Conv2D(filters, kernel_size=3, padding="same", strides=strides)
        self.bn0 = BatchNormalization()
        self.relu0 = ReLU()

        self.conv1 = Conv2D(filters, kernel_size=3, padding="same")
        self.bn1 = BatchNormalization()
        self.add = Add()
        self.relu1 = ReLU()
        if downsample and strides != 1:
          # self.downsample = Conv2D(filters, kernel_size=1, strides=strides)
          self.downsample = Sequential([Conv2D(filters, kernel_size=1, strides=strides),
                                        BatchNormalization()])

        else:
          self.downsample = Layer() # identity layer

    def call(self, inputs, *args, **kwargs):
        x = self.conv0(inputs)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1(x)
        x = self.bn1(x)

        identity = self.downsample(inputs)
        output = self.add([identity, x])

        output = self.relu1(output)
        return output

class ResNet18(Model):

    def __init__(self, num_classess=5):
        super().__init__()
        self.conv1 = Conv2D(64, 7, strides=2, padding="same")
        self.bn1 = BatchNormalization()
        self.relu1 = ReLU()
        self.pool1 = MaxPooling2D(3, strides=2, padding="same")

        num_block = 2

        # Longer version of code
        conv2_x_block = []
        for idx_b in range(num_block):
          conv2_x_block.append(BasicBlock_withResidule(filters=64, strides=1, downsample=False))
        self.conv2_x = Sequential(conv2_x_block)

        conv3_x_block = []
        for idx_b in range(num_block):
          conv3_x_block.append(BasicBlock_withResidule(filters=128, strides=2 if idx_b == 0 else 1, downsample=True if idx_b == 0 else False))
        self.conv3_x = Sequential(conv3_x_block)

        conv4_x_block = []
        for idx_b in range(num_block):
          conv4_x_block.append(BasicBlock_withResidule(filters=256, strides=2 if idx_b == 0 else 1, downsample=True if idx_b == 0 else False))
        self.conv4_x = Sequential(conv4_x_block)

        conv5_x_block = []
        for idx_b in range(num_block):
          conv5_x_block.append(BasicBlock_withResidule(filters=512, strides=2 if idx_b == 0 else 1, downsample=True if idx_b == 0 else False))
        self.conv5_x = Sequential(conv5_x_block)

        # Shorter version of code
        # self.conv2_x = Sequential([BasicBlock_withResidule(filters=64, strides=1, downsample=False)])
        # self.conv3_x = Sequential([BasicBlock_withResidule(filters=128, strides=2 if idx_b == 0 else 1, downsample=True if idx_b == 0 else False) for idx_b in range(num_block)])
        # self.conv4_x = Sequential([BasicBlock_withResidule(filters=256, strides=2 if idx_b == 0 else 1, downsample=True if idx_b == 0 else False) for idx_b in range(num_block)])
        # self.conv5_x = Sequential([BasicBlock_withResidule(filters=512, strides=2 if idx_b == 0 else 1, downsample=True if idx_b == 0 else False) for idx_b in range(num_block)])


        self.global_pool = GlobalAvgPool2D()
        self.fc = Dense(5)

        self.build(input_shape=(None, 224, 224, 3))
        self.call(Input(shape=(224, 224, 3)))

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)

        x = self.global_pool(x)
        x = self.fc(x)

        return x








In [ ]:
# TODO - Compile the model with optimizer, loss and metrics

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Keras offers plenty of callbacks function
from tensorflow.keras.callbacks import ReduceLROnPlateau

epochs = 50
resnet_18 = ResNet18()
resnet_18.compile(optimizer="adam",
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])

# reduce learning rate when the loss is not decreasing after 3 continuous epochs
reduce_lr_callback = ReduceLROnPlateau(
    monitor="loss", factor=0.5, patience=3, verbose=2, min_lr=1e-8
)
history = resnet_18.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[reduce_lr_callback])